In [3]:
import sqlite3
conn = sqlite3.connect("restaurants.db")
with open("restaurants.sql", "r") as f:
    sql_script = f.read()
conn.executescript(sql_script)
conn.commit()

In [4]:
conn = sqlite3.connect("restaurants.db")
tables = conn.execute(
    "SELECT name FROM sqlite_master WHERE type='table';"
).fetchall()
print(tables)

[('restaurants',)]


In [7]:
import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

conn = sqlite3.connect("restaurants.db")
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

orders_users = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

fnl_dataset = pd.merge(
    orders_users,
    restaurants,
    how="left",
    on="restaurant_id"
)

fnl_dataset.to_csv("final_dataset.csv", index=False)

print("Final dataset created successfully!")

Final dataset created successfully!


In [8]:
fnl_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [10]:
fnl_dataset.shape

(10000, 12)

In [14]:
fnl_dataset.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [15]:
import numpy as np

In [19]:
df = pd.read_csv("final_food_delivery_dataset.csv")

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9677 entries, 0 to 9676
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           9677 non-null   int64  
 1   user_id            9677 non-null   int64  
 2   restaurant_id      9677 non-null   int64  
 3   order_date         9677 non-null   object 
 4   total_amount       9677 non-null   float64
 5   restaurant_name_x  9677 non-null   object 
 6   name               9677 non-null   object 
 7   city               9677 non-null   object 
 8   membership         9677 non-null   object 
 9   restaurant_name_y  9676 non-null   object 
 10  cuisine            9676 non-null   object 
 11  rating             9676 non-null   float64
dtypes: float64(2), int64(3), object(7)
memory usage: 907.3+ KB


In [32]:
gold_df = df[df["membership"] == "Gold"]

In [33]:
city_revenue = (
    gold_df
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)
print(city_revenue)

city
Chennai      1049768.30
Pune          976042.38
Bangalore     960581.83
Hyderabad     866793.10
Name: total_amount, dtype: float64


In [34]:
highest_city = city_revenue.idxmax()
highest_revenue = city_revenue.max()

print(highest_city, highest_revenue)

Chennai 1049768.3


In [35]:
lowest_city = city_revenue.idxmin()
lowest_revenue = city_revenue.min()

print(lowest_city, lowest_revenue)

Hyderabad 866793.1


In [36]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)

cuisine
Mexican    807.293124
Italian    800.623134
Chinese    798.245650
Indian     796.486836
Name: total_amount, dtype: float64

In [37]:
user_total = df.groupby("user_id")["total_amount"].sum()

user_total[user_total > 1000].count()

np.int64(2505)

In [41]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[0, 2, 3, 4, 5],
    labels=["3.0 – 3.5", "3.6 – 4.0", "4.1 – 4.5", "4.6 – 5.0"]
)
df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18104\4016613887.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6 – 5.0    4029276.35
4.1 – 4.5    3472543.56
3.6 – 4.0     246165.28
3.0 – 3.5          0.00
Name: total_amount, dtype: float64

In [40]:
df.rating.min()

3.0

In [43]:
df[df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False)

city
Chennai      811.258346
Hyderabad    804.075232
Bangalore    794.525914
Pune         780.209736
Name: total_amount, dtype: float64

In [46]:
rest_count = df.groupby("cuisine")["restaurant_id"].nunique()

In [47]:
revenue = df.groupby("cuisine")["total_amount"].sum()

In [48]:
combined = pd.concat([rest_count, revenue], axis=1)
combined.columns = ["restaurant_count", "total_revenue"]

combined.sort_values(["restaurant_count", "total_revenue"],
                      ascending=[True, False])

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1867894.82
Italian,126,1971934.78
Indian,126,1900417.59
Mexican,128,2007738.00


In [50]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = round((gold_orders / total_orders) * 100)
percentage

50

In [57]:
rest_stats = df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

rest_stats[rest_stats["total_orders"] < 20] \
    .sort_values("avg_order_value", ascending=False) \
    .head(1)

,avg_order_value,total_orders
restaurant_name_x,,
Sri Mess Punjabi,1072.433636,11


In [56]:
df.restaurant_name_y

0       Restaurant_450
1       Restaurant_309
2       Restaurant_107
3       Restaurant_224
4       Restaurant_293
             ...      
9672    Restaurant_167
9673    Restaurant_459
9674    Restaurant_348
9675    Restaurant_490
9676               NaN
Name: restaurant_name_y, Length: 9677, dtype: object

In [59]:
df.tail()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range
9672,9673,2064,167,14-02-2023,876.97,Grand Tiffins Non-Veg,User_2064,Hyderabad,Regular,Restaurant_167,Italian,4.7,4.6 – 5.0
9673,9674,2838,459,13-06-2023,249.78,Sai Delights Non-Veg,User_2838,Hyderabad,Gold,Restaurant_459,Italian,4.6,4.6 – 5.0
9674,9675,1496,348,03-12-2023,975.77,Royal Restaurant Andhra,User_1496,Chennai,Regular,Restaurant_348,Italian,3.0,3.6 – 4.0
9675,9676,7,490,13-06-2023,1251.21,Swagath Curry House Punjabi,User_7,Chennai,Gold,Restaurant_490,Italian,3.7,4.1 – 4.5
9676,9677,1571,442,27-11-2023,993.02,Sai Eatery Andhra,User_1571,Pune,Gol,NaN,NaN,NaN,NaN


In [60]:
df.groupby(["city", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False) \
  .head(1)


city       cuisine
Bangalore  Italian    554108.75
Name: total_amount, dtype: float64

In [62]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False) \
  .head(5)

membership  cuisine
Regular     Mexican    1033205.66
            Italian     993764.03
Gold        Italian     978170.75
            Mexican     974532.34
            Chinese     955219.80
Name: total_amount, dtype: float64

In [63]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")
df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_18104\99252249.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


quarter
2023Q3    1976738.02
2023Q4    1956648.60
2023Q1    1924937.50
2023Q2    1874079.21
2024Q1      16574.88
Freq: Q-DEC, Name: total_amount, dtype: float64

In [65]:
gold_orders_count = df[df["membership"] == "Gold"]["order_id"].count()
gold_orders_count

np.int64(4832)

In [67]:
df[df["membership"] == "Gold"].shape[0]

4832

In [68]:
hyderabad_revenue = (
    df[df["city"] == "Hyderabad"]["total_amount"]
    .sum()
)

round(hyderabad_revenue)

1828091

In [69]:
distinct_users = df["user_id"].nunique()
distinct_users

2867

In [71]:
gold_avg_order_value = (
    df[df["membership"] == "Gold"]["total_amount"]
    .mean()
)

round(gold_avg_order_value, 2)

np.float64(797.43)

In [72]:
orders_high_rating = df[df["rating"] > 4.5]["order_id"].count()
orders_high_rating

np.int64(2676)

In [74]:
top_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

top_city

'Chennai'

In [75]:
orders_in_top_city = df[
    (df["membership"] == "Gold") &
    (df["city"] == top_city)
]["order_id"].count()

orders_in_top_city

np.int64(1294)

In [79]:
df.shape

(9677, 14)